<a href="https://colab.research.google.com/github/SemihDemir13/chatbot-project-data/blob/main/V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#1
print("Gerekli kütüphaneler kuruluyor...")
!pip install -q -U transformers huggingface_hub accelerate sentence-transformers torch langchain langchain-community langchain-huggingface chromadb pandas
print("Kurulum tamamlandı.")

Gerekli kütüphaneler kuruluyor...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 778.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
#2
from huggingface_hub import login

# Token'ını girdikten sonra 'Login successful' yazısını görmelisin.
login()

In [3]:
#3
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

print("Model yükleniyor... (Biraz sabır)")

model_id = "google/gemma-2b-it"

try:
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    # Colab GPU'su kullanılsın diye device_map="auto" diyoruz
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        torch_dtype="auto"
    )

    # Text generation pipeline'ı oluşturuyoruz
    llm_pipeline = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=250 # Cevabın uzunluğu
    )
    print("✅ Model başarıyla yüklendi!")

except Exception as e:
    print(f"❌ HATA: Model yüklenemedi. Lütfen HÜCRE 2'de token girdiğinizden ve modelin lisansını kabul ettiğinizden emin olun.\nHata: {e}")

Model yükleniyor... (Biraz sabır)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Device set to use cuda:0


✅ Model başarıyla yüklendi!


In [13]:
#4
import pandas as pd
import sys

print("CSV dosyası okunuyor ve RASTGELE seçiliyor...")

try:
    df = pd.read_csv('netflix_titles.csv')
    df = df.fillna('') # Boş yerleri doldur

    dokumanlar = []

    # DÜZELTME: head(2000) yerine sample(n=2000) kullanıyoruz.
    # Böylece listenin başından değil, karışık her yerinden veri alıyoruz.
    # Eğer hata alırsan n=1000 yapabilirsin.
    df_sample = df.sample(n=2000, random_state=42)

    for index, row in df_sample.iterrows():
        metin = (
            f"Title: {row['title']}\n"
            f"Type: {row['type']}\n"
            f"Director: {row['director']}\n"
            f"Cast: {row['cast']}\n"
            f"Country: {row['country']}\n"
            f"Release Year: {row['release_year']}\n"
            f"Rating: {row['rating']}\n"
            f"Duration: {row['duration']}\n"
            f"Categories: {row['listed_in']}\n"
            f"Description: {row['description']}"
        )
        dokumanlar.append(metin)

    print(f"✅ {len(dokumanlar)} adet içerik RASTGELE seçildi ve işlendi.")

except FileNotFoundError:
    print("❌ HATA: 'netflix_titles.csv' bulunamadı!")

CSV dosyası okunuyor ve RASTGELE seçiliyor...
✅ 2000 adet içerik RASTGELE seçildi ve işlendi.


In [14]:
#5
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

print("Vektör veritabanı (Çok Dilli) oluşturuluyor...")

# DEĞİŞİKLİK BURADA:
# Artık Türkçe-İngilizce eşleştirmesi yapabilen 'multilingual' modeli kullanıyoruz.
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# ChromaDB veritabanını kur
# Eğer hata alırsan önceki veritabanını temizlemek gerekebilir ama genellikle üzerine yazar.
vektor_veritabani = Chroma.from_texts(dokumanlar, embedding=embedding_model)

print("✅ Çok dilli veritabanı hazır! Artık Türkçe sorguları İngilizce verilerle eşleştirebilirim.")

Vektör veritabanı (Çok Dilli) oluşturuluyor...
✅ Çok dilli veritabanı hazır! Artık Türkçe sorguları İngilizce verilerle eşleştirebilirim.


In [17]:
def rag_yanit(soru, top_k=2): # DİKKAT: Sayıyı 2'ye düşürdük, kafa karışıklığını önlemek için.
    # 1. En alakalı 2 içeriği bul
    aranan_dokumanlar = vektor_veritabani.similarity_search(soru, k=top_k)

    # DEBUG: Hangi filmleri bulduğunu görelim (Sadece senin görmen için)
    print(f"\n[SİSTEM: Veritabanından Çekilenler]")
    context_list = []
    for i, doc in enumerate(aranan_dokumanlar):
        # İçerikten satır satır verileri ayıkla
        lines = doc.page_content.split('\n')
        title = lines[0].replace("Title: ", "")
        print(f"{i+1}. {title}")
        context_list.append(doc.page_content)
    print("-" * 30)

    # 2. Bulunan içerikleri birleştir
    baglam = "\n###\n".join(context_list)

    # 3. GÜÇLENDİRİLMİŞ PROMPT
    # Modele adım adım ne yapacağını söylüyoruz.
    prompt = f"""Sen yardımcı bir asistansın. Görevin, aşağıda verilen 'FİLM BİLGİLERİ' kısmını kullanarak kullanıcının sorusuna uygun bir film önermektir.

KURALLAR:
1. Sadece aşağıda verilen FİLM BİLGİLERİ içindeki filmlerden birini seç.
2. Asla dışarıdan bilgi uydurma.
3. Seçtiğin filmin adını ve konusunu Türkçe olarak özetle.

FİLM BİLGİLERİ:
{baglam}

KULLANICI SORUSU: {soru}

CEVAP (Şu formatta yaz: "Önerim: [Film Adı]. Konusu: [Açıklama]"):"""

    # 4. Modeli çalıştır
    sonuc = llm_pipeline(prompt)[0]["generated_text"]

    # 5. Prompt kısmını temizle, sadece cevabı al
    # Model bazen promptu tekrar edebilir, o yüzden "CEVAP" etiketinden sonrasını alıyoruz.
    if "CEVAP" in sonuc:
        cevap = sonuc.split("CEVAP")[-1].strip()
        # Bazen (Şu formatta yaz...) kısmını da tekrar eder, onu da temizleyelim
        if '):' in cevap:
            cevap = cevap.split('):')[-1].strip()
    else:
        cevap = sonuc

    return cevap

In [18]:
print("\n================================================")
print("🎬 NETFLIX GURUSU (Çıkmak için 'çıkış' yazın)")
print("================================================\n")

while True:
    soru = input("Soru sor: ")
    if soru.lower() in ['çıkış', 'exit', 'quit']:
        print("İyi seyirler 👋")
        break

    print("🔎 Araştırıyorum...")
    cevap = rag_yanit(soru)
    print(f"\n🤖 AI: {cevap}\n")
    print("-" * 40)


🎬 NETFLIX GURUSU (Çıkmak için 'çıkış' yazın)

Soru sor: aksiyon filmi öner
🔎 Araştırıyorum...

[SİSTEM: Veritabanından Çekilenler]
1. What Lies Below
2. The 2nd
------------------------------

🤖 AI: Önerim: "The 2nd". Konusu, aksiyon ve akışımsız şekilde geçen filmin başlığının ve konusu hakkında bilgi edineceğiniz böyle bir filmin önerimidir.

----------------------------------------
Soru sor: komedi filmi öner
🔎 Araştırıyorum...

[SİSTEM: Veritabanından Çekilenler]
1. Fun with Dick & Jane
2. The Stand-Up
------------------------------

🤖 AI: Önerim: Fun with Dick & Jane. Filmin konusu, iki yüksek düzeyde işletmenin kaybından sonra başlayan bir paralı cinsi bir kaçımın hikayesidir.

----------------------------------------
Soru sor: 2010 yapımı bir film öner
🔎 Araştırıyorum...

[SİSTEM: Veritabanından Çekilenler]
1. Uncorked
2. Seven Pounds
------------------------------

🤖 AI: Önerim, "Seven Pounds" filmini önermektedir. Film, 2008 yılında yapılmış ve Will Smith, Rosario Dawson, Woo

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


🔎 Araştırıyorum...

[SİSTEM: Veritabanından Çekilenler]
1. Free to Play
2. Coded Bias
------------------------------

🤖 AI: Önerim, Free to Play filminde yer alan "International Championship of Competitive Gaming" isimli filmin yönetmenliğini nolan bir şekilde yapılmış olduğunu ve bu nedenle filmin konu hakkında bilgi edici olduğunu düşünüyor.

----------------------------------------
Soru sor: christopher nolan ın yönetmenliğini yaptığı bir film öner
🔎 Araştırıyorum...

[SİSTEM: Veritabanından Çekilenler]
1. Inception
2. Catch Me If You Can
------------------------------

🤖 AI: Önerim: Inception. Film, başkaların uykusuzluklarını ve psikolojilerinin anlaşılmasıyla ilgili önemli bir filmdır. Filmlerin başrollerinde Leonardo DiCaprio ve Joseph Gordon-Levitt rol alır.

----------------------------------------
Soru sor: çıkış
İyi seyirler 👋
